# File formats

In principle, Niimpy can deal with any files of any format - you only need to convert them to a DataFrame.  Still, it is very useful to have some common formats, so we present two standard formats with default readers:

* **CSV files** are very standard and normal to create and understand, but in order to deal with them everything must be loaded into memory.
* **sqlite3 databases**, which requires sqlite3 to read, but provides more power for filtering and automatic processing without reading everything into memory.
* **Google TakeOut** provides a large selection of data in different formats. We provide readers most commonly used data types.
* **MHealth** is a common format for health data.

## DataFrame format (in-memory)

In-memory, data is stored in a [pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).  This is basically a normal dataframe.  There are some standardized columns (see the [schema](schema.html)) and the index is a DatetimeIndex.

## CSV files

CSV files should have a header that lists the column names and generally be readable by `pandas.read_csv`.

Reading these can be done with `niimpy.read_csv`:

In [ ]:
import os
import niimpy 
import niimpy.config as config

# Read the battery data
df= niimpy.read_csv(config.MULTIUSER_AWARE_BATTERY_PATH, tz='Europe/Helsinki')

## sqlite3 databases

For the purposes of niimpy, sqlite3 databases can generally be seen as supercharged CSV files.

A single database file could contain multiple datasets within it, thus when reading them a **table name** must be specified.

One reads the entire database into memory using `sqlite.read_sqlite`:

In [ ]:
# Read the sqlite3 data
df= niimpy.read_sqlite(config.SQLITE_SINGLEUSER_PATH, table="AwareScreen", tz='Europe/Helsinki')

You can list the tables within a database using `niimpy.reading.read.read_sqlite_tables`:

In [ ]:
niimpy.reading.read.read_sqlite_tables(config.SQLITE_SINGLEUSER_PATH)

sqlite3 files are highly recommended as a data storage format, since many common exploration options can be done within the database itself without reading the whole data into memory or writing an iterator.  However, the interface is more difficult to use.  Niimpy (before 2021-07) used this as its primary interface, but since then this interface has been de-emphasized.  You can read more in [the database section](database.html), but this is only recommended if you need efficiency when using massive amounts of data.

## Google TakeOut

Google takeout contains a many different types of data and new types are added as Google creates services or changes data storage methods. Readers are currently available for location data, emails, and activity data from the fit app. For other data types, the user needs to manually convert them into a Niimpy compatible Pandas DataFrame.

In [ ]:
# Data downloaded from Google Takeout is compressed as a zip archive to conserve disk space. To 
# demonstrate reading for the zip file, we will first compress our example data into the zip format.
import zipfile
test_zip = zipfile.ZipFile("test.zip", mode="w")

for dirpath,dirs,files in os.walk(config.GOOGLE_TAKEOUT_DIR):
    for f in files:
        filename = os.path.join(dirpath, f)
        filename_in_zip = filename.replace(config.GOOGLE_TAKEOUT_DIR, "")
        test_zip.write(filename, filename_in_zip)

test_zip.close()


In [ ]:
# Next we read location data from the zip file.
import niimpy
import niimpy.config as config
import niimpy.preprocessing.location as nilo

data = niimpy.reading.google_takeout.location_history("test.zip")
data = nilo.filter_location(
    data,
    latitude_column = "latitude",
    longitude_column = "longitude",
    remove_disabled=False, remove_network=False, remove_zeros=True
)
data


Activity data is read similarly. The data contains many columns with missing data, so in order to use the step count data, for example, we must set the NaN values to 0. 

In [ ]:
data = niimpy.reading.google_takeout.activity("test.zip")
data.loc[data["step_count"].isna(), "step_count"] = 0
data[["calories_(kcal)", "step_count"]]

The `google_takeout.email_activity` and `google_takeout.chat` function will read and process all emails in the GMail mailbox and all Google chat messages respectively. They return a dataframe containing metadata and statistics of each message. Email addresses, email IDs and names are replaced by numerical indexes.

The email files can be large and processing them could take some time. You can also include sentiment analysis of each email using the `sentiment` parameter. For this, we recommend using a system with a GPU.

In [ ]:
niimpy.reading.google_takeout.email_activity("test.zip")

In [ ]:
niimpy.reading.google_takeout.chat("test.zip", sentiment=True)

Finally, we have a reader for extracting Youtube watch history data. We do not, by default, return video identifiers, but replace them with numerical IDs. The only available information then is the recorded time, which corresponds to video start time.

Importantly, we have no information on how long the user watched a given video, as this is not stored in the TakeOut data. You can deduce whether the user has rewatched a given video, watched multiple videos in a row, or started another video quickly without finishing the previous one.

In [ ]:
niimpy.reading.google_takeout.youtube_watch_history("test.zip")

Since Google takeout may provide the mailbox as a single uncompressed file, it is also possible to provide it's file path directly.

In [ ]:
path = os.path.join(config.GOOGLE_TAKEOUT_DIR, "Takeout", "Mail", "All mail Including Spam and Trash.mbox")
niimpy.reading.google_takeout.email_activity(path, sentiment=True)

Each subject Downloads their Google TakeOut data as a separate zip file. The Zipfile package, which is included in the Python standard, is convenient for reading the data files contained in the zip file. For example, one could read the location data with the following code:

In [ ]:
from zipfile import ZipFile
import json
import pandas as pd

zip_file = ZipFile("test.zip")
json_data  = zip_file.read("Takeout/Location History (Timeline)/Records.json")
json_data = json.loads(json_data)
data = pd.json_normalize(json_data["locations"])
data = pd.DataFrame(data)
data.head()

Location data is stored in the json format. Other types of data are stored in various formats and with different files structures. The user must find how each type of data they need is stored and how it can be read in Python.

## MHealth

We have implemented readers for 3 data types formatted according to the [MHealth schema](https://www.openmhealth.org/documentation/#/schema-docs/schema-library). These are total sleep time, heart rate and geolocation. Other data types may be added as needed.

In [ ]:
# Reading total sleep time data:
filename = config.MHEALTH_TOTAL_SLEEP_TIME_PATH
niimpy.reading.mhealth.total_sleep_time_from_file(filename)

In [ ]:
# Reading heart rate data:
filename = config.MHEALTH_HEART_RATE_PATH
niimpy.reading.mhealth.heart_rate_from_file(filename)

In [ ]:
# Reading geolocation data:
filename = config.MHEALTH_GEOLOCATION_PATH
niimpy.reading.mhealth.geolocation_from_file(filename)

In [1]:
import niimpy.reading.google_takeout as takeout

zip_filename = "../data/takeout-jarno-001.zip"
datatypes = takeout.fit_list_data(zip_filename)
datatypes = datatypes[
    datatypes["content"].str.contains("step_count.delta") &
    datatypes["source"].str.contains("polarflow")
]
filename = datatypes["filename"].iloc[0]

data = takeout.fit_read_data(zip_filename, filename)

/u/24/rantahj1/unix/miniconda3/envs/niimpy/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
data

array([  0,   3,   1,   2,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  16,  15,  17,  18,  19,  20,  21,  23,  22,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  65,
        64,  72,  66,  67,  68,  69,  70,  71,  73,  74,  75,  76,  77,
        83,  78,  79,  80,  81,  82,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  98,  97,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 126, 125, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 138, 137, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 156, 153, 157,
       155, 154, 159, 160, 158, 161, 164, 163, 162, 165, 166, 167, 168,
       169, 171, 170, 172])

## Other formats

You can add readers for any types of formats which you can convert into a Pandas dataframe (so basically anything).  For examples of readers, see `niimpy/reading/read.py`.  Apply the function `niimpy.preprocessing.util.df_normalize` in order to apply some standardizations to get the standard Niimpy format.